## Описание проекта

Из открытых источников доступны исторические данные о продажах игр интернет-магазина «Стримчик», который продаёт по всему миру компьютерные игры,
оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation).
Нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными.
Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.
В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр.
ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».


## Цель проекта

Выявить определяющие успешность игры закономерности, с целью сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

### Описание данных

•	Name — название игры
•	Platform — платформа
•	Year_of_Release — год выпуска
•	Genre — жанр игры
•	NA_sales — продажи в Северной Америке (миллионы проданных копий)
•	EU_sales — продажи в Европе (миллионы проданных копий)
•	JP_sales — продажи в Японии (миллионы проданных копий)
•	Other_sales — продажи в других странах (миллионы проданных копий)
•	Critic_Score — оценка критиков (максимум 100)
•	User_Score — оценка пользователей (максимум 10)
•	Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.


## Шаг 1. Загрузим данные

In [3]:
import pandas as pd
import numpy as np
import re
import plotly
import plotly.graph_objs as go
import plotly.express as px
from scipy import stats as st

% matplotlib inline

UsageError: Line magic function `%` not found.


In [5]:
# загрузим данные
data = pd.read_csv('../data/games.csv')
# дадим датафрейму имя для дальнейшего удобства использования
data.name = 'games'
# преобразуем названия колонок в нижний регистр
data.columns = [x.lower() for x in data.columns.to_list()]
# выведем датафрейм и информацию по нему на экран
display(
    data.head(),
    data.describe(),
    data.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


,year_of_release,na_sales,eu_sales,jp_sales,other_sales,critic_score
count,16446.000000,16715.000000,16715.000000,16715.000000,16715.000000,8137.000000
mean,2006.484616,0.263377,0.145060,0.077617,0.047342,68.967679
std,5.877050,0.813604,0.503339,0.308853,0.186731,13.938165
min,1980.000000,0.000000,0.000000,0.000000,0.000000,13.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,60.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,71.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,79.000000
max,2016.000000,41.360000,28.960000,10.220000,10.570000,98.000000


None

Датафрейм состоит из 16715 строк и 11 колонок, 6 из которых типа float и 5 типа Object.
В данных содержится информация по играм, выпущенным с 1980 по 2016 год, с оценками критиков от 13 до 98.
Посмотрим на пропуски в данных

In [6]:
def get_missing_values(data: pd.DataFrame) -> None:
    """
    Выводит данные о пропусках в колонках по датафрейму.
    Не изменяет данные внутри датафрейма.

    :param data: pd.DataFrame
    :return: None
    """
    # получаем имена колонок датафрейма
    columns = data.columns.to_list()
    data_len = len(data)
    # объявляем счетчик
    counter = -1
    display('='*60)
    # если есть пропуски в данных - выводим информацию о пропусках по колонкам
    if sum(data.isnull().sum()) > 0:
        display(f'Количество записей в датафрейме {data.name}: {data_len}')
        display(f'В датафрейме {data.name} имеются следующие пропуски:')
        for i in data.isnull().sum():
            counter += 1
            if i > 0:
                display(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
    else:
        display(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(data)

'============================================================'

'Количество записей в датафрейме games: 16715'

'В датафрейме games имеются следующие пропуски:'

'  - в колонке name: 2 пропусков, это 0.01% об общего объема данных'

'  - в колонке year_of_release: 269 пропусков, это 1.61% об общего объема данных'

'  - в колонке genre: 2 пропусков, это 0.01% об общего объема данных'

'  - в колонке critic_score: 8578 пропусков, это 51.32% об общего объема данных'

'  - в колонке user_score: 6701 пропусков, это 40.09% об общего объема данных'

'  - в колонке rating: 6766 пропусков, это 40.48% об общего объема данных'

У нас есть 2 пропуска в колонке name и genre. Скорее всего они в одних и тех же строках, убедимся в этом.

In [7]:
data[data['name'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Да, оба пропуска находятся в одинаковых строках. Удалим их.

In [8]:
data.drop(index=[659, 14244], inplace=True)
# убедимся, что строки удалены
data[data['name'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


Также видно, что в названиях некоторых игр есть год. Обычно год в названии игры совпадает с годом ее релиза,
а значит мы можем восстановить часть пропусков в колонке year_of_release спарсив из названия игры год релиза.

In [9]:
def search(x: pd.Series) -> int:
    """
    Функция для поиска даты релиза игры в названии

    :param x: Series вида ['name', 'year_of_release']
    :type x: pd.Series
    :return: Год релиза игры
    :rtype: int
    """
    name = x['name']
    year = x['year_of_release']

    if pd.isna(year):
        try:
            match = re.search(r'[1-2][0-9][0-9][0-9]', name).group()
            if match:
                return match
        except AttributeError:
            return year
    else:
        return year

# посчитаем количество пропусков до восстановления
before = len(data[data['year_of_release'].isna()])
# восстановим пропуски в колонке
data['year_of_release'] = data[['name','year_of_release']].apply(search, axis=1)
# посчитаем количество пропусков после восстановления
after = len(data[data['year_of_release'].isna()])

display(f"Количество восстановленных данных: {before - after}")

'Количество восстановленных данных: 17'

Год релиза это точно целое число, так что преобразуем данную колонку из float16 в int16

In [11]:
data['year_of_release'] = pd.to_numeric(data['year_of_release']).convert_dtypes()
# проверим, что код отработал верно
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16461 non-null  Int64  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: Int64(1), float64(5), object(5)
memory usage: 1.5+ MB


Преобразуем типы данных

In [12]:
def auto_change_dtypes(data: pd.DataFrame) -> None:
    """
    Автоматически определяет тип столбца, и изменяет его в соответствии с хранимыми значениями.
    Функция не возвращает новый датафрейм, а изменяет переданный в качестве аргумента.
    Функция заточена под данные конкретного проекта.
    Функция поддерживает автоматическое преобразование следующих типов и форматов данных:
     - int64
     - float64
     - str: если в названии колонки есть date и формат даты %Y-%m-%d,
            то переводит в формат pandas datetime, иначе - переводит строковые данные в нижний регистр

    Пример преобразования:
    data[column] является int64 и содержит значения в диапазоне от 0 до 100 - будет преобразован в int8
    data[date_column] является object и содержит в имени колонки date - будет преобразован в datetime

    :param data: pd.DataFrame
    :return: None
    """
    # получаем количество используемой датафреймом памяти
    memory_usage_before_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    # получаем описание датафрейма
    describe = data.describe()
    # получаем названия колонок
    columns = data.columns.to_list()
    # получаем типы данных
    dtypes = data.dtypes
    # количество типов данных
    indexes = len(dtypes)
    # создаем 2 словаря для int и float, содержащие в качестве ключей типы данных, а значений - список из min и max значений этих типов данных
    correct_int_dtypes = {'int8': [-2**7, 2**7-1], 'int16': [-2**15, 2**15-1], 'int32': [-2**31, 2**31-1]}
    correct_float_dtypes = {'float16': [-2.0**16, 2.0**16-1], 'float32': [-2.0**31, 2.0**31-1]}

    display(f'{"="*30} Работаем с датафреймом {data.name} {"="*30}')
    # пробегаем по индексам колонок датафрейма и типам данных колонок
    for index, dtype in zip(range(0, indexes), dtypes):

        # если тип int64, меняем на тип, соответствующий значениям колонок
        if dtype == np.int64:
            for key, value in zip(correct_int_dtypes.keys(), correct_int_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип float64, меняем на тип, соответствующий значениям колонок
        elif dtype == np.float64:
            for key, value in zip(correct_float_dtypes.keys(), correct_float_dtypes.values()):
                if not describe[columns[index]]['min'] <= value[0] and not describe[columns[index]]['max'] >= value[1]:
                    display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на {key}')
                    data[columns[index]] = data[columns[index]].astype(key)
                    break

        # если тип object и колонка содержит в названии 'date' - меняем на datetime
        elif dtype == object:
            if 'date' in columns[index]:
                display(f'Изменяем тип колонки {columns[index]} датафрейма {data.name} с {dtype} на datetime')
                data[columns[index]] = pd.to_datetime(data[columns[index]], format='%Y-%m-%d')
            # иначе приводим данные к нижнему регистру, пропуская значения nan (float) в колонке
            else:
                display(f'Приводим строковые данные в колонке {columns[index]} к нижнему регистру')
                data[columns[index]] = data[columns[index]].apply(lambda s: s if type(s) == float else s.lower())

    # количество памяти, используемое датафреймом оптимизации типов данных
    memory_usage_after_change_dtypes = data.memory_usage(index=False, deep=True).sum()
    bytes_in_mb = 2**23
    display(f'Использование памяти датафрейма до сжатия: {(memory_usage_before_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Использование памяти датафрейма после сжатия: {(memory_usage_after_change_dtypes / bytes_in_mb):.2f} мб.')
    display(f'Сжато: {((memory_usage_before_change_dtypes - memory_usage_after_change_dtypes) / bytes_in_mb):.2f} мб.')

auto_change_dtypes(data)

'============================== Работаем с датафреймом games =============================='

'Приводим строковые данные в колонке name к нижнему регистру'

'Приводим строковые данные в колонке platform к нижнему регистру'

'Приводим строковые данные в колонке genre к нижнему регистру'

'Изменяем тип колонки na_sales датафрейма games с float64 на float16'

...

'Приводим строковые данные в колонке user_score к нижнему регистру'

'Приводим строковые данные в колонке rating к нижнему регистру'

'Использование памяти датафрейма до сжатия: 0.70 мб.'

'Использование памяти датафрейма после сжатия: 0.64 мб.'

'Сжато: 0.06 мб.'

k-a = E  - с 1998 года, так что переименовываем k-a в E

Индексы 475, 488, 594 - повторы

In [69]:
# data[['name', 'year_of_release']].apply(lambda x, y: re.search('[1-2][0-9][0-9][0-9]', x['name']).group(), y['year_of_release'] if re.search('[1-2][0-9][0-9][0-9]', x['name']) else 0, axis=1)

def t(row):
    if re.search('[1-2][0-9][0-9][0-9]', row['name']):
        match = int(re.search('[1-2][0-9][0-9][0-9]', row['name']).group())
        if match != int(row['year_of_release']):
            return row['name'], row['year_of_release']

data[['name', 'year_of_release']].apply(t, axis=1).dropna().T


,0
206,"(warzone 2100, 1999)"
238,"(madden nfl 2005, 2004)"
250,"(winning eleven: pro evolution soccer 2007, 2006)"
283,"(madden nfl 2003, 2002)"
333,(mario & sonic at the london 2012 olympic game...
...,...
16528,"(pro evolution soccer 2010, 2009)"
16551,"(winning eleven: pro evolution soccer 2007, 2006)"
16585,"(farming 2017 - the simulation, 2016)"
16709,(score international baja 1000: the official g...
